In [5]:
import asyncio
import nest_asyncio
from playwright.async_api import async_playwright

# Jupyter Notebook で `asyncio.run()` のエラーを回避
nest_asyncio.apply()

async def test_playwright():
    async with async_playwright() as p:
        browser = await p.chromium.launch()
        print("Playwright is working correctly!")
        await browser.close()

# 直接 `asyncio.run()` を使わずに動作確認
asyncio.get_event_loop().run_until_complete(test_playwright())


Playwright is working correctly!


In [37]:
import asyncio
from TikTokApi import TikTokApi

async def fetch_tiktok_videos():
    async with TikTokApi(session_id="verify_m7nbzp87_hEwzHL08_GI7E_4pYF_ARcR_jOoFxcjMB0Je") as api:
        hashtag_name = "fashion"
        tag = api.hashtag(name=hashtag_name)

        videos = tag.videos(count=5)  # 5件の動画を取得

        async for video in videos:
            print(f"Video ID: {video.id}, Likes: {video.stats['diggCount']}")

asyncio.run(fetch_tiktok_videos())



TypeError: __init__() got an unexpected keyword argument 'session_id'

In [16]:
from TikTokApi import TikTokApi

# session_id を設定
SESSION_ID = "84ac2af4bda2b4ec06f19c939e96b55c"

def fetch_tiktok_user_videos():
    api = TikTokApi()  # `5.2.2` では `session_id` を `__init__()` に渡さない
    api.session.cookies.set("sessionid", SESSION_ID)  # `session_id` をセット

    username = "tiktok"  # 取得したいユーザー名
    user = api.user(username=username)

    videos = user.videos(count=5)  # 5件の動画を取得

    for video in videos:
        print(f"Video ID: {video['id']}, Likes: {video['stats']['diggCount']}")

fetch_tiktok_user_videos()


AttributeError: 'TikTokApi' object has no attribute 'session'

In [38]:
import asyncio
from playwright.async_api import async_playwright

async def scrape_tiktok_hashtag(hashtag: str):
    async with async_playwright() as p:
        # ヘッドレスブラウザを起動
        browser = await p.chromium.launch(headless=True)
        # 日本語向けの設定を追加
        context = await browser.new_context(
            user_agent="Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/115.0.0.0 Safari/537.36",
            viewport={"width": 1280, "height": 720},
            locale="ja-JP",
            extra_http_headers={"Accept-Language": "ja-JP,ja;q=0.9"}
        )
        page = await context.new_page()
        
        # URLに言語パラメータを追加
        url = f"https://www.tiktok.com/tag/{hashtag}?lang=ja-JP"
        print("Navigating to:", url)
        await page.goto(url, wait_until="networkidle")
        
        # ページ上のCookie同意などのポップアップがあれば処理（必要ならクリック）
        # 例: await page.click("button[aria-label='同意する']")
        
        # 十分に待機して動画が読み込まれるのを待つ
        await asyncio.sleep(10)
        
        # 複数回スクロールして無限スクロールのコンテンツをロード
        for _ in range(3):
            await page.evaluate("window.scrollTo(0, document.body.scrollHeight)")
            await asyncio.sleep(3)
        
        # 動画リンクを含む要素を抽出（セレクタは必要に応じて調整）
        video_elements = await page.query_selector_all("a[href*='/video/']")
        print(f"Found {len(video_elements)} video elements.")
        
        for video in video_elements:
            video_link = await video.get_attribute("href")
            print("動画リンク:", video_link)
        
        await browser.close()

asyncio.run(scrape_tiktok_hashtag("古着"))



Navigating to: https://www.tiktok.com/tag/古着?lang=ja-JP
Found 0 video elements.


In [36]:


from TikTokApi import TikTokApi
# s_v_web_idをverifyFpに入れる
verifyFp='verify_m7nbzp87_hEwzHL08_GI7E_4pYF_ARcR_jOoFxcjMB0Je'
api = TikTokApi.get_instance(custom_verifyFp=verifyFp, use_test_endpoints=True)

# 取得数
count = 1
# ハッシュタグ
hashtag = '古着'
result = api.by_hashtag(count=count, hashtag=hashtag)

for t in result:
    print('----------------------------------')
    print('ユーザーID : ' + t['author']['uniqueId'])
    print('ユーザー名　: ' + t['author']['nickname'])
    print('----------------------------------')

AttributeError: type object 'TikTokApi' has no attribute 'get_instance'